In [479]:
import pandas as pd
import numpy as np
import time
import random
import math
import xgboost
from sklearn.preprocessing import LabelEncoder
from sklearn import ensemble, preprocessing, metrics
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
import category_encoders as ce
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix
from random import shuffle
from sklearn.model_selection import KFold
%autosave 120

Autosaving every 120 seconds


In [442]:
sample_submission = pd.read_csv('./sample_submission.csv')
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
t = train['is_claim']
train = train.drop(['is_claim'], axis = 1)
tmp = pd.concat([train, test], axis = 0)
tmp = tmp.drop(['policy_id'], axis = 1)
labelencoder = LabelEncoder()
for i in tmp.columns[:-1] : 
    if(tmp[i].dtype == object):
        tmp[i] = labelencoder.fit_transform(tmp[i])
tmp['turning_radius'] = tmp['turning_radius'] * 100
tmp['turning_radius'] = tmp['turning_radius'].astype(int)

# 連續變數離散化
tmp1 = tmp.copy()
a = tmp1.columns
b = 22
c = []
    
for i in range(tmp1.shape[1]) :
    if(tmp1[a[i]].dtype != object) :
        if(len(tmp1[a[i]].unique()) >= b) :
            tmp1[a[i]] = pd.cut(tmp1[a[i]], b)
            tmp1[a[i]] = tmp[a[i]].astype(int)
            c.append(a[i])

train1 = tmp1.iloc[:train.shape[0], ::]
train1 = pd.concat([train1, t], axis = 1)
test1 = tmp1.iloc[train.shape[0]+1:, ::]
print(sample_submission.shape)
print(train1.shape)
# print(train.columns)
print(test1.shape)
print(train1.head(1))
print(c)
print(train1.info())

(39063, 2)
(58592, 43)
(39062, 42)
   policy_tenure  age_of_car  age_of_policyholder  area_cluster  \
0              0           0                    0             0   

   population_density  make  segment  model  fuel_type  max_torque  ...  \
0                4990     1        0      0          0           5  ...   

   is_brake_assist  is_power_door_locks  is_central_locking  \
0                0                    0                   0   

   is_power_steering  is_driver_seat_height_adjustable  \
0                  1                                 0   

   is_day_night_rear_view_mirror  is_ecw  is_speed_alert  ncap_rating  \
0                              0       0               1            0   

   is_claim  
0         0  

[1 rows x 43 columns]
['policy_tenure', 'age_of_car', 'age_of_policyholder', 'area_cluster', 'population_density']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 58592 entries, 0 to 58591
Data columns (total 43 columns):
 #   Column                        

## Naive Bayes

### Bayes method1

In [510]:
def accuracy_score(y, y_pred) : 
    r = sum(y == y_pred)/len(y)
    return r

In [138]:
#Calcualting probabilites for inputs independantly  p(A)
def prior(data, col):
    t = data[col]
    t = t.value_counts()
    return t/data.shape[0]

In [139]:
#calculating conditional probability  p(A|B)
def posterior(data, want, given):
    ab = data[[want, given]]
    a = data[want].unique()
    b = data[given].unique()
    
    groups = ab.groupby(by = [given, want]).size().reset_index()
    a = pd.DataFrame(data[given].value_counts()[groups[given]]).reset_index()
    groups = pd.concat([groups, a], axis = 1)
    groups.columns = ['B', 'A', 'P(A|B)', 'i', 'B_total']
    groups['P(A|B)'] = groups['P(A|B)']/groups.B_total
    groups = groups.drop(['i', 'B_total'], axis = 1)

    return  groups

In [499]:
def Naive_Bayes(x_train, x_test, out):
    ans = []
    y_train = x_train[out]
    y_prob = prior(x_train, out)
    y = x_train[out].unique()
    test_ans = x_test[out]
    test = x_test.drop([out], axis = 1)
    for s in range(len(test)) :
        t = test.iloc[s, ::]
        for i in y :
            each = []
            e = y_prob[y]
            for j in x_train.columns[:-1] :
                tmp = posterior(x_train, j, out)
                idx1 = tmp['A'] == t[j]
                idx2 = tmp['B'] == i
                tmp = tmp[idx1 & idx2]
                if (tmp.empty):
                    tmp = 0
                else : 
                    tmp = tmp.iloc[0,2] 
                e = e * tmp
            each.append(e)
        a = np.argmax(e)
        ans.append(y[a])
    acc = sum( test_ans == ans )
    
    return acc, ans

In [163]:
# 9hr
s = time.time()
train2 = train1.iloc[:3000, ::]
a = round(len(train2) * 0.7)
w = random.sample(range(len(train2)), a)
b = list(range(len(train2)))
w1 = [a - b for a, b in zip( list(range(len(train2))), w)]
tr = train2.iloc[w]
te = train2.iloc[w1]
acc, ans = Naive_Bayes(tr, te, 'is_claim')
print(time.time() - s)

973.5264370441437


### Bayes method2

In [512]:
class Naive_Bayes :
    def __init__(self) :
        self.feats = list
        self.likelihoods = {}
        self.class_priors = {}
        self.pred_priors = {}
        self.x = np.array
        self.y = np.array
        self.nsample = int
        self.nfeats = int

    def fit(self, x, y) :
        self.x = x
        self.y = y
        self.feats = list(x.columns)
        self.nsample = x.shape[0]
        self.nfeats = x.shape[1]

        for i in self.feats :
            self.likelihoods[i] = {}
            self.pred_priors[i] = {}
            for j in np.unique(self.x[i]) :
                self.pred_priors[i].update({j : 0})
                for k in np.unique(self.y) :
                    self.likelihoods[i].update({str(j) + '_' + str(k) : 0})
                    self.class_priors.update({k : 0})
                
        self.cal_class_prior()
        self.cal_likelihoods()
        self.cal_predictor_prior()
        
    def cal_class_prior(self) : # P(A)
        for i in np.unique(self.y) :
            self.class_priors[i] = sum(self.y == i)/self.nsample
    
    def cal_likelihoods(self) : # P(A|B)
         for feature in self.feats:
            for outcome in np.unique(self.y):
                outcome_count = sum(self.y == outcome)
                feat_likelihood = self.x[feature][self.y[self.y == outcome].index.values.tolist()].value_counts().to_dict()

                for feat_val, count in feat_likelihood.items():
                    if (outcome_count != 0) : 
                        self.likelihoods[feature][str(feat_val) + '_' + str(outcome)] = count/outcome_count
                    else : 
                        self.likelihoods[feature][str(feat_val) + '_' + str(outcome)] = 0
        
    def cal_predictor_prior(self):
        for feature in self.feats:
            feat_vals = self.x[feature].value_counts().to_dict()

            for feat_val, count in feat_vals.items():
                self.pred_priors[feature][feat_val] = count/self.nsample   
        
        
    def predict(self, x) :
        result = []
        x = np.array(x)
        for query in x : 
            probs_outcome = {}
            for out in np.unique(self.y) :
                prior = self.class_priors[out]
                likelihood = 1
                e = 1
                for i, j in zip(self.feats, query) :
                    t = str(j) + '_' + str(out)
                    likelihood *= self.likelihoods[i][t]
                    e *= self.pred_priors[i][j]
                if (e != 0) : 
                    posterior = (likelihood * prior) / e
                else : 
                    posterior = 0
                probs_outcome[out] = posterior
            r = max(probs_outcome, key = lambda a : probs_outcome[a])
            result.append(r)
        return np.array(result)

In [513]:
s = time.time()
a = round(len(train1) * 0.7)
random.seed(1)
w = random.sample(range(len(train1)), a)
b = list(range(len(train1)))
w1 = [a - b for a, b in zip( list(range(len(train1))), w)]
tr = train1.iloc[w, ::]
te = train1.iloc[w1, ::]

a = Naive_Bayes()
a.fit(tr.iloc[::, :-1], tr.iloc[::, -1])
p = a.predict(te.iloc[::, :-1])
acc = accuracy_score(p, te.iloc[::, -1]) 

print(acc)
print(p)
print(sum(p))
print(time.time() - s)

0.9325352318720437
[0 0 0 ... 0 0 0]
107
29.863337516784668


## Decision Tree

In [373]:
# self
'''
Note that you can implement the simplifiedversion of the random forest
the full function is unnecessary in this assignment. 
Basically, you are required to implement the bagging in #data and feature dimension with the predefined ratio. 
The other internal hyperparameters of the #tree, #max-depth of each tree, etc., could be predefined by
yourself, just mentioned in your report.

1. bagging in #data
2. internal hyperparameters of the #tree, #max-depth of each tree

'''

class RandomForestClassifier :
    def __init__(self) -> None:
        pass

    
    def train_test_split(self, data, test) : 
        a = data.shape[0]
        train_size = round(a*test)
        train_index = random.sample(range(a), b) 
        test_index = [a - b for a, b in zip( list(range(len(data))), w)]
        train_df = data.iloc[train_index, ::]
        test_df = data.iloc[test_index, ::]
        
        return train_df, test_df
    
    # 1. Decision Tree helper functions
    # 1.1 Data pure?
    def check_purity(self, data) : # 確認資料是否不用再被切分
        y = data[:, -1]
        unique_classes = np.unique(y)
        if len(unique_classes) == 1 :
            return True
        else : 
            return False
    
    # 1.2 Classify    
    def create_leaf(self, data) : # 找最大的類別
        label_column = data[:, -1]
        unique_classes, counts_unique_classes = np.unique(label_column, return_counts = True)
        index = counts_unique_classes.argmax()
        leaf = unique_classes[index]
        
        return leaf
    
    # 1.3 Determine potential splits
    def get_potential_split(self, data) : # 列出所有出現過的變數種類
        potential_splits = {}
        n_columns = data.shape[1]
        for i in range(n_columns - 1) :
            values = data[::, i]
            unique_values = np.unique(values)
            potential_splits[i] = unique_values
            
        return potential_splits
    
    # 1.4 Determine Best Split
    def cal_entropy(self, data) : # 計算cross entropy
        label_column = data[:, -1]
        counts = np.unique(label_column, return_counts=True)[1]
        probabilities = counts / counts.sum()
        e = sum(probabilities * -np.log2(probabilities))

        return e
    
    def cal_overall_metrix(self, data_below, data_above) : # 看該次分組的成效?
        n = len(data_below) + len(data_above)
        p_data_below = len(data_below) / n
        p_data_above = len(data_above) / n

        overall_metric =  (p_data_below * self.cal_entropy(data_below) + p_data_above * self.cal_entropy(data_above))

        return overall_metric
    
    def determine_best_split(self, data, potential_splits) : # 找 要切的變數 跟 要切的值
        best_overall_metric = np.inf # 自己加的     
        for column_index in potential_splits:
            for value in potential_splits[column_index]:
                data_below, data_above = self.split_data(data, split_column=column_index, split_value=value)
                current_overall_metric = self.cal_overall_metrix(data_below, data_above)
                if current_overall_metric <= best_overall_metric:
                    best_overall_metric = current_overall_metric
                    best_split_column = column_index
                    best_split_value = value

        return best_split_column, best_split_value

    # 1.5 Split data
    def split_data(self, data, split_column, split_value): # 切資料

        split_column_values = data[:, split_column]

        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            data_below = data[split_column_values <= split_value]
            data_above = data[split_column_values >  split_value]

        # feature is categorical   
        else:
            data_below = data[split_column_values == split_value]
            data_above = data[split_column_values != split_value]

        return data_below, data_above
    
    # 2. Decision Tree Algorithm
    # 2.1 Helper Function
    def determine_type_of_feature(self, df): # 判斷變數是 連續的 還是 離散的
        feature_types = []
        n_unique_values_threshold = 15
        for feature in df.columns:
            if feature != 'is_claim':
                unique_values = df[feature].unique()
                example_value = unique_values[0]

                if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_threshold):
                    feature_types.append("categorical")
                else:
                    feature_types.append("continuous")

        return feature_types

    # 2.2 Algorithm
    def decision_tree_algorithm(self, df, counter=0, min_samples=2, max_depth=5):

        # data preparations
        if counter == 0:
            global COLUMN_HEADERS, FEATURE_TYPES
            COLUMN_HEADERS = df.columns
            FEATURE_TYPES = self.determine_type_of_feature(df)
            data = df.values
        else:
            data = df           

        # base cases
        if (self.check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
            leaf = self.create_leaf(data)
            return leaf


        # recursive part
        else:    
            counter += 1

            # helper functions 
            potential_splits = self.get_potential_split(data)
            split_column, split_value = self.determine_best_split(data, potential_splits)
            data_below, data_above = self.split_data(data, split_column, split_value)

            # check for empty data
            if len(data_below) == 0 or len(data_above) == 0:
                leaf = self.create_leaf(data)
                return leaf

            # determine question
            feature_name = COLUMN_HEADERS[split_column]
            type_of_feature = FEATURE_TYPES[split_column]
            if type_of_feature == "continuous":
                question = "{} <= {}".format(feature_name, split_value)

            # feature is categorical
            else:
                question = "{} = {}".format(feature_name, split_value)

            # instantiate sub-tree
            sub_tree = {question: []}

            # find answers (recursion)
            yes_answer = self.decision_tree_algorithm(data_below, counter, min_samples, max_depth)
            no_answer = self.decision_tree_algorithm(data_above, counter, min_samples, max_depth)

            # If the answers are the same, then there is no point in asking the qestion.
            # This could happen when the data is classified even though it is not pure
            # yet (min_samples or max_depth base case).
            if yes_answer == no_answer:
                sub_tree = yes_answer
            else:
                sub_tree[question].append(yes_answer)
                sub_tree[question].append(no_answer)

            return sub_tree

    # 3. Make predictions
    # 3.1 One example
    def predict_example(self, example, tree):

        # tree is just a root node
        if not isinstance(tree, dict):
            return tree

        question = list(tree.keys())[0]
        feature_name, comparison_operator, value = question.split(" ")

        # ask question
        if comparison_operator == "<=":
            if example[feature_name] <= float(value):
                answer = tree[question][0]
            else:
                answer = tree[question][1]

        # feature is categorical
        else:
            if str(example[feature_name]) == value:
                answer = tree[question][0]
            else:
                answer = tree[question][1]

        # base case
        if not isinstance(answer, dict):
            return answer

        # recursive part
        else:
            residual_tree = answer
            return self.predict_example(example, residual_tree)


    # 3.2 All examples of a dataframe
    def make_predictions(self, df, tree):
        predictions = df.apply(self.predict_example, args=(tree,), axis=1)
        return predictions

    def bootstrap(self, data, n) : 
        index = np.random.randint(0, len(data), n)
        df = data.iloc[index, ::]
        
        return df

    def random_forest_algorithm(self, train, n_trees, n_bootstrap, n_features, dt_max_depth) :
        forest = []
        for i in range(n_trees) : 
            df_bootstrap = self.bootstrap(train, n_bootstrap)
            tree = self.decision_tree_algorithm(df_bootstrap, max_depth=dt_max_depth)
            forest.append(tree)
            
        return forest
    
    def random_forest_predictions(self, test, forest) :
        pred = {}
        for i in range(len(forest)) : 
            column = 'tree{}'.format(i)
            p = self.make_predictions(test, tree=forest[i])
            pred[column] = p
            
        pred = pd.DataFrame(pred)
        rf = pred.mode(axis = 1)[0]
        
        return rf

'''        
        
    # 3.3 Accuracy
    def calculate_accuracy(self, df, tree):
        predictions = make_predictions(df, tree)
        predictions_correct = predictions == df.label
        accuracy = predictions_correct.mean()

        return accuracy
''' 


'        \n        \n    # 3.3 Accuracy\n    def calculate_accuracy(self, df, tree):\n        predictions = make_predictions(df, tree)\n        predictions_correct = predictions == df.label\n        accuracy = predictions_correct.mean()\n\n        return accuracy\n'

In [398]:
s = time.time()
train2 = train1
a = round(len(train2) * 0.7)
random.seed(123)
w = random.sample(range(len(train2)), a)
tr = train2.iloc[w, ::]
te = train2.drop(w)

rf = RandomForestClassifier()
r = rf.random_forest_algorithm(tr, n_trees = 100, n_bootstrap = 3000, n_features = 5, dt_max_depth = 10)
result = rf.random_forest_predictions(te, r )
    
print(time.time() - s)

250.83877420425415


In [399]:
print(len(te))
print(len(result))
print(sum(result))
acc = accuracy_score(result, te.iloc[::, -1]) 
print(acc)
print(sum(result))

17578
17578
0
0.9342359767891683
0


## Random Forest Classifier (scikit-learning)

In [420]:
X_train, X_test, y_train, y_test = train_test_split(train.iloc[::, :-1], train.iloc[::, -1], test_size = 0.3)

# 建立 random forest 模型
forest = ensemble.RandomForestClassifier(n_estimators = 100)
forest_fit = forest.fit(X_train, y_train)
# 預測
test_y_predicted = forest.predict(X_test)
# 績效
accuracy = metrics.accuracy_score(y_test, test_y_predicted)
print('Random Forest Classifier Package Accuracy : ', accuracy)

0.9199567641369895


## XGBoost

In [434]:
X_train, X_test, y_train, y_test = train_test_split(train.iloc[::, :-1], train.iloc[::, -1], test_size = 0.3)
# 建立 XGBClassifier 模型
xgboostModel = XGBClassifier() # , enable_categorical = True
# 使用訓練資料訓練模型
xgboostModel.fit(X_train, y_train)
# 使用訓練資料預測分類
predicted = xgboostModel.predict(X_train)
# 預測成功的比例
print('訓練集: ',xgboostModel.score(X_train,y_train))
print('測試集: ',xgboostModel.score(X_test,y_test))

訓練集:  0.9351440971375627
測試集:  0.9381044487427466


## CatBoost

In [459]:
X_train, X_test, y_train, y_test = train_test_split(train1.iloc[::, :-1], train1.iloc[::, -1], test_size = 0.3)
X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train, test_size = 0.1)
clf = CatBoostClassifier(iterations=2000, eval_metric = 'Accuracy', verbose = 500)
# Fit model
clf.fit(X_train, y_train, eval_set = (X_eval, y_eval))
# Make predictions
pred = clf.predict(X_test)
acc = accuracy_score(pred, y_test) 
print('CatBoost Package Accuracy : ', acc)

Learning rate set to 0.057155
0:	learn: 0.9370665	test: 0.9351536	best: 0.9351536 (0)	total: 27.4ms	remaining: 54.7s
500:	learn: 0.9371749	test: 0.9341784	best: 0.9351536 (0)	total: 9.13s	remaining: 27.3s
1000:	learn: 0.9372020	test: 0.9336909	best: 0.9351536 (0)	total: 18.5s	remaining: 18.4s
1500:	learn: 0.9372020	test: 0.9336909	best: 0.9351536 (0)	total: 27.6s	remaining: 9.19s
1999:	learn: 0.9372020	test: 0.9336909	best: 0.9351536 (0)	total: 36.3s	remaining: 0us

bestTest = 0.9351535836
bestIteration = 0

Shrink model to first 1 iterations.
CatBoost Package Accuracy :  0.9340653089088633


## LightGBM

In [462]:
X_train, X_test, y_train, y_test = train_test_split(train1.iloc[::, :-1], train1.iloc[::, -1], test_size = 0.3)
classifier = lgb.LGBMClassifier(objective = 'binary', 
                                learning_rate = 0.05, 
                                n_estimators = 100, 
                                random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

print(cm)

[[16460     0]
 [ 1118     0]]


In [480]:
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4])
kf = KFold(n_splits=3)
kf.get_n_splits(X)
print(kf)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


KFold(n_splits=3, random_state=None, shuffle=False)
TRAIN: [2 3] TEST: [0 1]
TRAIN: [0 1 3] TEST: [2]
TRAIN: [0 1 2] TEST: [3]


##　Cross Validation

In [514]:
k = [3, 5, 10]
for i in k : 
    # k fold
    random.seed(1)
    kf = KFold(n_splits=i)
    t = np.array(train1)
    kf.get_n_splits(t[::, :-1])
    nb = []
    ranfor = []
    for train_index, test_index in kf.split(t[::, :-1]):
        
        X_train, X_test = t[::, :-1][train_index], t[::, :-1][test_index]
        y_train, y_test = t[::, -1][train_index], t[::, -1][test_index]
        
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)
        y_train = pd.DataFrame(y_train)
        y_test = pd.DataFrame(y_test)
        
        
        # self naive bayes
        # a = Naive_Bayes()
        # a.fit(X_train, y_train)
        # p = a.predict(X_test)
        # acc = accuracy_score(p, np.array(y_test)) 
        # nb.append(acc)
        
        # self random forest
        rf = RandomForestClassifier()
        r = rf.random_forest_algorithm(pd.concat([X_train, y_train], axis = 1), n_trees = 100, n_bootstrap = 3000, n_features = 5, dt_max_depth = 10)
        result = rf.random_forest_predictions(pd.concat([X_test, y_test], axis = 1), r )
        
        ranfor.append(result)
    print('Self Random Forest ', i, ' fold cross validation : ', ranfor)
    print('Self Random Forest ', i, ' fold cross validation : ', ranfor.mean())
    
        # self random forest
    

AttributeError: 'DataFrame' object has no attribute 'unique'

In [398]:
# self rf 250s
s = time.time()
a = round(len(train1) * 0.7)
random.seed(123)
w = random.sample(range(len(train1)), a)
tr = train1.iloc[w, ::]
te = train1.drop(w)

rf = RandomForestClassifier()
r = rf.random_forest_algorithm(tr, n_trees = 100, n_bootstrap = 3000, n_features = 5, dt_max_depth = 10)
result = rf.random_forest_predictions(te, r )
    
print(time.time() - s)

250.83877420425415


In [478]:
X_train, X_test, y_train, y_test = train_test_split(train1.iloc[::, :-1], train1.iloc[::, -1], test_size = 0.3)
k = [3, 5, 10]
for i in k : 
    # RandomForest
    scores = cross_val_score(ensemble.RandomForestClassifier(n_estimators = 100),train1.iloc[::, :-1], train1.iloc[::, -1], cv = i, scoring = 'accuracy')
    print('RandomForest ', i, ' fold cross validation : ', scores)
    print('RandomForest ', i, ' fold cross validation : ', scores.mean())
    
    # XGBoost
    scores = cross_val_score(XGBClassifier(),train1.iloc[::, :-1], train1.iloc[::, -1], cv = i, scoring = 'accuracy')
    print('XGboost ', i, ' fold cross validation : ', scores)
    print('XGboost ', i, ' fold cross validation : ', scores.mean())
    
    #Catboost
    scores = cross_val_score(CatBoostClassifier(iterations=2000, eval_metric = 'Accuracy', verbose = 500),train1.iloc[::, :-1], train1.iloc[::, -1], cv = i, scoring = 'accuracy')
    print('CatBoost ', i, ' fold cross validation : ', scores)
    print('CatBoost ', i, ' fold cross validation : ', scores.mean())
    
    # LightGBM
    scores = cross_val_score(lgb.LGBMClassifier(objective = 'binary', learning_rate = 0.05,  n_estimators = 100, random_state=0),train1.iloc[::, :-1], train1.iloc[::, -1], cv = i, scoring = 'accuracy')
    print('LightGBM ', i, ' fold cross validation : ', scores)
    print('LightGBM ', i, ' fold cross validation : ', scores.mean())
    
    

RandomForest  3  fold cross validation :  [0.93558958 0.93528237 0.93584229]
RandomForest  3  fold cross validation :  0.9355714136892446
XGboost  3  fold cross validation :  [0.93558958 0.93569198 0.9359959 ]
XGboost  3  fold cross validation :  0.9357591520472367
Learning rate set to 0.026095
0:	learn: 0.9360231	total: 34.7ms	remaining: 1m 9s
500:	learn: 0.9360231	total: 8.33s	remaining: 24.9s
1000:	learn: 0.9361255	total: 17.3s	remaining: 17.3s
1500:	learn: 0.9361255	total: 26.3s	remaining: 8.75s
1999:	learn: 0.9361255	total: 35.2s	remaining: 0us
Learning rate set to 0.026095
0:	learn: 0.9360487	total: 25.7ms	remaining: 51.4s
500:	learn: 0.9360487	total: 8.13s	remaining: 24.3s
1000:	learn: 0.9361255	total: 17.2s	remaining: 17.2s
1500:	learn: 0.9361255	total: 26.3s	remaining: 8.74s
1999:	learn: 0.9361255	total: 35.7s	remaining: 0us
Learning rate set to 0.026096
0:	learn: 0.9360248	total: 25.4ms	remaining: 50.7s
500:	learn: 0.9360248	total: 8.15s	remaining: 24.4s
1000:	learn: 0.936076